In [ ]:
# Support for TF2 models was added after TF 2.3.
!pip install tf-nightly

In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
# Download the checkpoint and put it into models/research/object_detection/test_data/

!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz
!tar -xf ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz

In [ ]:
#https://github.com/tensorflow/models/blob/master/research/object_detection/export_tflite_graph_tf2.py
%%bash
python models/research/object_detection/export_tflite_graph_tf2.py \
  --pipeline_config_path /content/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/pipeline.config \
  --trained_checkpoint_dir /content/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/checkpoint \
  --output_directory /content/tflite

In [ ]:
!tflite_convert --saved_model_dir=tflite/saved_model --output_file=tflite/model.tflite

In [ ]:
from google.colab import files
files.download('tflite/model.tflite')

In [ ]:
import tensorflow as tf

In [ ]:
# Tflite model: dynamic range quantization
converter = tf.lite.TFLiteConverter.from_saved_model('tflite/saved_model')
converter.experimental_new_converter = False
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model_dynamic = converter.convert()

tflite_models_dir = pathlib.Path("tflite")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the model:
tflite_model_file = tflite_models_dir/"dynamic_model.tflite"
tflite_model_file.write_bytes(tflite_model_dynamic)

In [ ]:
from google.colab import files
files.download('tflite/dynamic_model.tflite')

In [ ]:
# Tflite model: float 16 quantization
# For debugging
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

converter = tf.lite.TFLiteConverter.from_saved_model('tflite/saved_model')
converter.experimental_new_converter = False
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

tflite_model_16_float = converter.convert()

tflite_models_dir = pathlib.Path("tflite")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the model:
tflite_model_file = tflite_models_dir/"16_float.tflite"
tflite_model_file.write_bytes(tflite_model_16_float)

In [ ]:
from google.colab import files
files.download('tflite/16_float.tflite')

In [ ]:
import numpy as np

In [ ]:
# Tflite model: integer only quantization
# For debugging
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)
num_calibration_images = 100

# Random representative data
def representative_data_gen():
  for _ in range(num_calibration_images):
    image = np.random.randint(low=0, high=255, size=(1, 640, 640, 3))
    image = image.astype(np.float32)
    yield [image]

converter = tf.lite.TFLiteConverter.from_saved_model('tflite/saved_model')
converter.experimental_new_converter = False
converter.allow_custom_ops = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_int_rep_random = converter.convert()

tflite_models_dir = pathlib.Path("tflite")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the model:
tflite_model_file = tflite_models_dir/"8_int_model_rep_random.tflite"
tflite_model_file.write_bytes(tflite_model_int_rep_random)

In [ ]:
from google.colab import files
files.download("tflite/8_int_model_rep_random.tflite")

In [ ]:
# Tflite model: integer only quantization
# For debugging
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

converter = tf.lite.TFLiteConverter.from_saved_model('tflite/saved_model')
converter.experimental_new_converter = False
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model_int_rep_random = converter.convert()

tflite_models_dir = pathlib.Path("tflite")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the model:
tflite_model_file = tflite_models_dir/"default.tflite"
tflite_model_file.write_bytes(tflite_model_int_rep_random)

In [ ]:
from google.colab import files
files.download("tflite/default.tflite")